# 3) Calling Functions — Exercises

**Goals:** positional vs keyword calls, unpacking (`*`, `**`), higher-order functions (functions as values), callbacks, `key=` usage, lambdas vs named functions, error wrappers.

### Warm-ups

1. **Call with keywords**

```python
def rect(w, h): return w*h
def rect_kw(**kwargs):
    """Call rect using **kwargs expected to have w and h."""
    ...
assert rect_kw(w=3, h=4) == 12
```

2. \**Call with *args**

```python
def hypotenuse(a, b):
    return (a*a + b*b) ** 0.5
def hypot_from_pair(pair):
    """Call hypotenuse using *pair."""
    ...
assert abs(hypot_from_pair((3,4)) - 5) < 1e-9
```

3. **Sort with key**

```python
def sort_by_last_char(words):
    """Return new list sorted by last character; stable; do not mutate input."""
    ...
assert sort_by_last_char(["ab","aa","bb"]) == ["aa","ab","bb"]
```

### Core

4. **Apply pipeline**

```python
def apply_pipeline(x, funcs):
    """
    funcs: iterable of callables f: x -> x
    Return result of applying them in order.
    """
    ...
assert apply_pipeline(2, [lambda x: x+3, lambda x: x*10]) == 50
```

5. **Filter + map with callables**

```python
def transform(nums, pred, mapper):
    """Keep items where pred(x) is True, then map with mapper(x)."""
    ...
assert transform([1,2,3,4], lambda x:x%2==0, lambda x:x*x) == [4,16]
```

6. **Group with key function**

```python
def group_by(xs, key_fn):
    """Return dict key->list of items in original order."""
    ...
g = group_by(["aa","b","ccc","dd"], key_fn=len)
assert g == {2:["aa","dd"],1:["b"],3:["ccc"]}
```

7. **Retry wrapper**

```python
def retry(fn, *, attempts=3):
    """
    Call fn(); if it raises, retry up to attempts times.
    Return fn() result or raise last error.
    """
    ...
count = {"n":0}
def flaky():
    count["n"] += 1
    if count["n"] < 2: raise RuntimeError("boom")
    return "ok"
assert retry(flaky, attempts=3) == "ok"
```

8. **Decorator: timeit**

```python
import time
def timeit(fn):
    """Decorator returning (result, elapsed_seconds)."""
    ...
@timeit
def slow_add():
    t0 = time.time()
    while time.time() - t0 < 0.01: pass
    return 42
res, secs = slow_add()
assert res == 42 and secs >= 0
```

9. **Caching (simple memoize)**

```python
def memoize(fn):
    """Decorator caching fn(x) for hashable x."""
    ...
calls = {"n":0}
@memoize
def square(x):
    calls["n"] += 1
    return x*x
assert square(10)==100 and square(10)==100 and calls["n"]==1
```

### Challenge

10. **Compose callables**

```python
def compose(*funcs):
    """
    Return f(x) that applies funcs right-to-left: compose(f,g,h)(x) == f(g(h(x))).
    If no funcs, return identity.
    """
    ...
id_fn = compose()
assert id_fn(5) == 5
f = compose(lambda x:x+1, lambda x:x*2, lambda x:x-3)
assert f(10) == ((10-3)*2)+1
```


In [11]:
# 1) Call with keywords
def rect(w, h): return w*h

def rect_kw(**kwargs):
    """Call rect using **kwargs expected to have w and h."""
    return rect(**kwargs)

assert rect_kw(w=3, h=4) == 12

In [12]:
# 2) Call with *args
def hypotenuse(a, b):
    return (a*a + b*b) ** 0.5

def hypot_from_pair(pair):
    """Call hypotenuse using *pair."""
    return hypotenuse(*pair)

assert abs(hypot_from_pair((3,4)) - 5) < 1e-9

In [13]:
# 3) Sort with key
def sort_by_last_char(words):
    """Return new list sorted by last character; stable; do not mutate input."""
    return sorted(words, key=lambda w: w[-1] if w else "")

assert sort_by_last_char(["ab","aa","bb"]) == ["aa","ab","bb"]

In [14]:
# 4) Apply pipeline
def apply_pipeline(x, funcs):
    """
    funcs: iterable of callables f: x -> x
    Return result of applying them in order.
    """
    for f in funcs:
        x = f(x)
    return x

assert apply_pipeline(2, [lambda x: x+3, lambda x: x*10]) == 50

In [15]:
# 5) Filter + map with callables
def transform(nums, pred, mapper):
    """Keep items where pred(x) is True, then map with mapper(x)."""
    return [mapper(x) for x in nums if pred(x)]

assert transform([1,2,3,4], lambda x:x%2==0, lambda x:x*x) == [4,16]

In [16]:
# 6) Group with key function
def group_by(xs, key_fn):
    """Return dict key->list of items in original order."""
    out = {}
    for x in xs:
        out.setdefault(key_fn(x), []).append(x)
    return out

g = group_by(["aa","b","ccc","dd"], key_fn=len)
assert g == {2:["aa","dd"],1:["b"],3:["ccc"]}

In [17]:
# 7) Retry wrapper
def retry(fn, *, attempts=3):
    """
    Call fn(); if it raises, retry up to attempts times.
    Return fn() result or raise last error.
    """
    last_exc = None
    for _ in range(attempts):
        try:
            return fn()
        except Exception as e:
            last_exc = e
    # exhausted attempts
    raise last_exc

count = {"n":0}
def flaky():
    count["n"] += 1
    if count["n"] < 2: raise RuntimeError("boom")
    return "ok"

assert retry(flaky, attempts=3) == "ok"

In [18]:
# 8) Decorator: timeit
import time

def timeit(fn):
    """Decorator returning (result, elapsed_seconds)."""
    def wrapper(*args, **kwargs):
        t0 = time.time()
        res = fn(*args, **kwargs)
        elapsed = time.time() - t0
        return res, elapsed
    return wrapper

@timeit
def slow_add():
    t0 = time.time()
    while time.time() - t0 < 0.01:  # busy-wait ~10ms
        pass
    return 42

res, secs = slow_add()
assert res == 42 and secs >= 0

In [19]:
# 9) Caching (simple memoize)
def memoize(fn):
    """Decorator caching fn(x) for hashable x."""
    cache = {}
    def wrapper(x):
        if x in cache:
            return cache[x]
        y = fn(x)
        cache[x] = y
        return y
    return wrapper

calls = {"n":0}
@memoize
def square(x):
    calls["n"] += 1
    return x*x

assert square(10)==100 and square(10)==100 and calls["n"]==1

In [20]:
# 10) Compose callables
def compose(*funcs):
    """
    Return f(x) that applies funcs right-to-left: compose(f,g,h)(x) == f(g(h(x))).
    If no funcs, return identity.
    """
    if not funcs:
        return lambda x: x
    def composed(x):
        for f in reversed(funcs):
            x = f(x)
        return x
    return composed

id_fn = compose()
assert id_fn(5) == 5
f = compose(lambda x:x+1, lambda x:x*2, lambda x:x-3)
assert f(10) == ((10-3)*2)+1